# _SEMOPY_: an efficient alternative to _lavaan_
## Structural Equation Modeling using Python
### 24.08.2023
### Leonardo Zaggia - demo project

# Tutorial

What does `semopy` do?

1. Write down a model description in a user-friendly syntax<br>
2. Estimate model's parameters using a variety of objective functions<br>
3. Calculate numerous statistics and fit indices<br>
4. Estimate model's parameters in presence of ordinal variables<br>
5. A vast number of settings to fit a researcher's needs<br>
6. Fast and accurate

## Installation

`semopy` is available at PyPi and can be installed by typing the following line into the terminal. Let's do some homekeeping: <br><br>
- create a new virtual environment and activate it
    ```bash
    python3 -m venv .venv
    . .venv/bin/activate
    ```

- install `semopy` and scikit-learn (package needed by semopy to do extra fast calculations)
    ```bash
    pip install semopy
    pip install scikit-learn
    ```
Here you go, your python environment has been set up and you can start using `semopy`!


## Syntax

To specify SEM models, `semopy` uses syntax that resembles the popular lavaan in R.<br>
The syntax supports three operator symbols characterizing relationships between variables:

```
- `~` to specify the structural part                -> the regression operator
- `=~` to specify the measurement part              -> for latent variable definition
- `~~` to specify common variance between variables
``````

For example, let a linear equation in the structural part of an SEM model take the form:

```python
y = β1 x1 + β2 x2 + ε

Then, in `semopy` syntax it becomes:

```python
y ~ x1 + x2


Parameters `β1`, `β2` are to be estimated by `semopy`.<br>
In some cases, a user might want to fix some parameters to a particular value. <br>
In Structural Equation Modeling (SEM) this is called fixing the loading.<br>
This 
For instance,<br>
using SEM gergo, if we want `β1` to be our fixed loading only estimate the loading belonging to the second indicator `β2`,<br>
we can do it as follows:

```python
y ~ 1*x1 + x2


Likewise, if a latent variable `η` is explained by manifest variables `y1`, `y2`, `y3`, then in `semopy` syntax it can be written down this way:

```{code-cell} ipython3
eta =~ y1 + y2 + y3


It is also possible to specify a type of variable. If variable `x2` is ordinal, we can inform the package about it using a special operator `is`:

```{code-cell}python
x2 is ordinal


## Quickstart

The pipeline for working with SEM models in `semopy` consists of three steps:

1. Specifying a model
2. Loading a dataset into the model
3. Estimating parameters of the model.

The power of SEM lies in it's ability to model latent variables. There a few of ways to specify them in semopy, but the most straightforward one is to use the measurement operator =~. See how it's done in a classical "Holzinger-Swineford 1939" dataset, we will start with a `CFA` model:

In [1]:
# import sample data provided by semopy
from semopy.examples import holzinger39

# extract an already specified model and data
model_specification = holzinger39.get_model()
data = holzinger39.get_data()

# show the model and the first few columns of the data
print(model_specification)
data.head()

visual =~ x1 + x2 + x3
textual =~ x4 + x5 + x6
speed =~ x7 + x8 + x9


,id,sex,ageyr,agemo,school,grade,x1,x2,x3,x4,x5,x6,x7,x8,x9
1,1,1,13,1,Pasteur,7.0,3.333333,7.75,0.375,2.333333,5.75,1.285714,3.391304,5.75,6.361111
2,2,2,13,7,Pasteur,7.0,5.333333,5.25,2.125,1.666667,3.00,1.285714,3.782609,6.25,7.916667
3,3,2,13,1,Pasteur,7.0,4.500000,5.25,1.875,1.000000,1.75,0.428571,3.260870,3.90,4.416667
4,4,1,13,2,Pasteur,7.0,5.333333,7.75,3.000,2.666667,4.50,2.428571,3.000000,5.30,4.861111
5,5,2,12,2,Pasteur,7.0,4.833333,4.75,0.875,2.666667,4.00,2.571429,3.695652,6.30,5.916667


Here the model was already specified for us.<br>
<br>
Alternatively, we could have specified the model ourselves: <br>

```python

# specify the model
model_specification = """
        visual =~ x1 + x2 + x3          # visual latent variable -> identified/measured by 3 indicators
        text =~ x4 + x5 + x6            # text latent variable -> identified/measured by 3 indicators
        text ~ visual                   # text is predicted by visual
      """
```

But lets get back to the tutorial. <br>
It is now time to fit the model and see what kind of results we can observe. <br><br>

**Note**
To keep the model concise, there are some **default settings** in `semopy` which you should keep in mind:

1. To scale latent variables, the factor loading of the first indicator of each latent variable is fixed to 1.

2. Residual variances are estimated automatically. 

3. All exogenous latent variables are correlated by default. 

**All the default settings can be overridden and/or switched off by the user.**
But we will not do that here. <br>

In [2]:
from semopy import Model        

# We proceed by passing the model specification to the Model class
model = Model(model_specification)

# We can now fit the model to the data, almost too easy!
model.fit(data)

# Lastly, we inspect the results of the model
ins = model.inspect()
print(ins)


       lval  op     rval  Estimate  Std. Err    z-value   p-value
0        x1   ~   visual  1.000000         -          -         -
1        x2   ~   visual  0.554421  0.099727   5.559413       0.0
2        x3   ~   visual  0.730526   0.10918   6.691009       0.0
3        x4   ~  textual  1.000000         -          -         -
4        x5   ~  textual  1.113076  0.065392  17.021522       0.0
5        x6   ~  textual  0.926120  0.055425  16.709493       0.0
6        x7   ~    speed  1.000000         -          -         -
7        x8   ~    speed  1.179980  0.165045   7.149459       0.0
8        x9   ~    speed  1.082517  0.151354   7.152197       0.0
9     speed  ~~    speed  0.383377  0.086171   4.449045  0.000009
10    speed  ~~   visual  0.262135  0.056252   4.659977  0.000003
11    speed  ~~  textual  0.173603  0.049316   3.520223  0.000431
12   visual  ~~   visual  0.808310  0.145287   5.563548       0.0
13   visual  ~~  textual  0.408277  0.073527    5.55273       0.0
14  textua

### Understanding the output
Each row represents a `relationship` (path) between two variables in your SEM model. Let's break down the columns:<br>

- **lval**: This column likely refers to the "left value" or the variable on the left side of the equation. It represents the predictor variable in a regression-like equation.
<br><br>
- **op**: This column may indicate the operation symbol used in the equation. In SEM syntax, ~ usually represents the structural relationship between variables.
<br><br>
- **rval**: This column likely refers to the "right value" or the variable on the right side of the equation. It represents the dependent variable in a regression-like equation.
<br><br>
- **Estimate**: This column displays the estimated parameter value (regression coefficient) for the relationship between the predictor and dependent variables.
<br><br>
- **Std. Err**: This column shows the standard error associated with the estimated parameter. It gives an indication of the uncertainty or variability of the parameter estimate.
<br><br>
- **z-value**: This column displays the z-value, which is the ratio of the estimate to the standard error. It's used to assess the statistical significance of the estimate.
<br><br>
- **p-value**: This column shows the p-value associated with the parameter estimate. It indicates the probability of observing the estimate if there is truly no effect (null hypothesis). Lower p-values suggest statistical significance.
<br><br>

In the context of SEM, these values are crucial for interpreting the relationships between variables in your model. For instance, you might examine the p-values to determine which relationships are statistically significant. The estimate values show the strength and direction of the relationship, while the standard errors and z-values help you gauge the precision and significance of the estimates.



### Land Mark Reached!! ###
I know this must have been a lot, but you have reached a land mark. You have learned how to use the semopy package to conduct a CFA, and you have all the knowledge you need to conduct a SEM.

<br><br>
But before we move on, let's take a look at the model that we have just created.
Since data and model visualization are crucial part of the analysis pipeline.
Take the code that you find here below and plot the model that you have just created.
<br><br>

```python
# Plot the model
from semopy import semplot
path_2_figures_storage = 'sem_figures/'
figure_name = 'simple_CFA.pdf'
plot = semplot(model, filename=(path_2_figures_storage + figure_name))
plot.view()
```
How does it look?
It is now your turn to try it out, apply some changes to the model and try ot plot it again. A reminder in in order:
1. **Specify the model** -> you can add a combination of new structural paths, new measurement paths, new latent variables, new observed variables
2. **Turn your model into a Model object** -> model = Model(model_specification)
<br><br>



In [5]:
from semopy import semplot
path_2_figures_storage = 'sem_figures/'
figure_name = 'example_SEM.pdf'
model2 = """
    visual =~ x1 + x2 + x3
    textual =~ x4 + x5 + x6
    speed =~ x7 + x8 + x9
    textual ~ visual
    speed ~ visual
    speed ~ textual

"""
model2 = Model(model2)

# We can now fit the model to the data, almost too easy!
model2.fit(data)

plot = semplot(model2, filename=(path_2_figures_storage + figure_name))
plot.view()

'sem_figures/example_SEM.pdf'

**Tasks:**

Following the same approach described above, lets define a SEM model based on the `demo_semopy.txt` you can find in the data folder.<br>
We will use what we learned today about the `semopy` package to do this.
This example is less abstract and more directly related to psychology, I hope you will find it interesting :) 

1. Define four latent variables

   **WM** (working memory) -  indicated by **WMv, WMn, WMf**
   
   **SM** (secondary memory) -  indicated by **SMv, SMn, SMf**
   
   **Spd** (speed) - indicated by **Spd1, Spd2, Spd3, Spd4**
   
   **gf** (fluid intelligence) - indicated by **gfv, gfn, gff**

   (psss... we talked about this as `model specification`)

2. Create a CFA model following Task 1. Apply both ways of scaling latent variables. Report the covariances between the latent factors.

3. Following Task _1_, create an SEM model based on your theoretical thinking on how these latent variables to relate or explain each other. Create a diagram of your postulated model.

In [ ]:
# import necessary function
from pandas import read_csv

# elements within txt files are usually 'tab' separated, 
# we need to pass that information to our function-> sep='\t'
data = read_csv('data/demo_semopy.txt', sep='\t')

# print the head of the data -> first 5 rows
print(data.head())
# print the keys of the df -> get an idea of what we are dealing with
print(data.keys())

model = Model(mod)
"""
# fit the model
res = model.fit(data)

# print the results
print(res.summary())
"""

## Hurray! <br>
### You have successfully learned how to move your SEM workflow from R to Python!!.
 I know Structural Equation Modeling can be intimidating, but with more hands on an practice, you will absorbe all the skills needed in __no time!__. You have succesfully reached the `end of this tutorial`, I hope you have learned something new and useful. If you have any questions, please feel free to drop me an email. I will try to answer them as soon as possible. Also, if you have any suggestions or feedback, please let me know. I would love to hear them. Thank you for reading this tutorial. Have a good day!